In [0]:
%run ../env

In [0]:
# Display sample records from historical_telephonic_outreach table
display(spark.table("historical_telephonic_outreach").limit(10))

In [0]:
# Show distribution of historical telephonic outreach by call_outcome
df = spark.table("historical_telephonic_outreach")
distribution_df = df.groupBy("call_outcome").count()
display(distribution_df)

In [0]:
# Display sample records where call_outcome is 'connected-completed'
df = spark.table("historical_telephonic_outreach")
sample_df = df.filter(df.call_outcome == "Connected - Completed").limit(10)
display(sample_df)

In [0]:
# Sample 10 member_ids from historical_telephonic_outreach
df = spark.table("historical_telephonic_outreach")
sample_member_ids = df.select("member_id").distinct().limit(30)
display(sample_member_ids)

In [0]:
# Filter the entire table to show only records for the 10 sampled member_ids
member_ids_list = [row.member_id for row in sample_member_ids.collect()]
filtered_df = df.filter(df.member_id.isin(member_ids_list))

print(f"Total records for {len(member_ids_list)} member_ids: {filtered_df.count()}")
display(filtered_df)

In [0]:
# Show distribution of call_outcomes for the filtered member_ids
outcome_distribution = filtered_df.groupBy("call_purpose").count().orderBy("count", ascending=False)
display(outcome_distribution)

In [0]:
# Create lookup table mapping healthcare to telco call purposes
telco_lookup = [
    ("Chronic Care Management", "5G Upgrade Opportunity"),
    ("Transportation Assistance", "Device Trade-In Program"),
    ("Health Risk Assessment", "Plan Optimization Review"),
    ("Benefits Education", "Family Plan Expansion"),
    ("Medication Adherence", "International Roaming Package"),
    ("Care Gap Closure", "Device Protection Plan"),
    ("Social Services Referral", "Data Overage Prevention"),
    ("Post-Discharge Follow-up", "Loyalty Rewards Redemption"),
    ("Annual Wellness Visit Scheduling", "New Service Bundle Offer")
]

# Create DataFrame from lookup table
telco_lookup_df = spark.createDataFrame(telco_lookup, ["healthcare_purpose", "telco_purpose"])
display(telco_lookup_df)

In [0]:
# Join filtered_df with lookup table to swap call purposes
from pyspark.sql.functions import coalesce

filtered_df_telco = filtered_df.join(
    telco_lookup_df, 
    filtered_df.call_purpose == telco_lookup_df.healthcare_purpose, 
    "left"
).withColumn(
    "call_purpose", 
    coalesce(telco_lookup_df.telco_purpose, filtered_df.call_purpose)
).drop("healthcare_purpose", "telco_purpose")

print(f"Total records with telco purposes: {filtered_df_telco.count()}")
display(filtered_df_telco)

In [0]:
from pyspark.sql.functions import row_number, concat, lpad, lit, col
from pyspark.sql.window import Window

# Get distinct agent_ids from filtered_df_telco
distinct_agents = filtered_df_telco.select("agent_id").distinct().orderBy("agent_id")

# Create fake names for 5 reps
fake_names = [
    "Sarah Mitchell",
    "James Rodriguez",
    "Emily Chen",
    "Michael Thompson",
    "Jessica Martinez"
]

# Collect agent_ids and create mapping with both rep_id and rep_name
agent_list = [row.agent_id for row in distinct_agents.collect()]
rep_mapping = [(agent_id, f"REP{str(i % 5 + 1).zfill(8)}", fake_names[i % len(fake_names)]) for i, agent_id in enumerate(agent_list)]

# Create DataFrame from mapping
rep_mapping_df = spark.createDataFrame(rep_mapping, ["agent_id", "rep_id", "rep_name"])
display(rep_mapping_df)

In [0]:
# Join rep mapping to filtered_df_telco
filtered_df_telco_with_reps = filtered_df_telco.join(
    rep_mapping_df,
    on="agent_id",
    how="left"
)

print(f"Total records with rep info: {filtered_df_telco_with_reps.count()}")
display(filtered_df_telco_with_reps)

In [0]:
# Show distribution of call_outcomes for the filtered member_ids
outcome_distribution = filtered_df.groupBy("call_outcome").count().orderBy("count", ascending=False)
display(outcome_distribution)

In [0]:
# Generate fake transcripts for connected calls and voicemails using AI_Query
from pyspark.sql.functions import col, concat, lit, when

# Filter for Connected outcomes and Voicemail Left
transcript_df = filtered_df_telco_with_reps.filter(
    (col("call_outcome").like("%Connected%")) | (col("call_outcome") == "Voicemail Left")
)

# Create the SQL query with ai_query
transcript_query = """
SELECT 
  call_id,
  member_id,
  rep_id,
  rep_name,
  call_date,
  call_time,
  call_outcome,
  call_purpose,
  call_duration_seconds,
  ai_query(
    'databricks-gpt-5-2',
    CASE 
      WHEN call_outcome = 'Voicemail Left' THEN
        CONCAT(
          'Generate a realistic voicemail message from a Telco call center representative named ', rep_name, ' calling on behalf of organization "Brickster Wireless". ',
          'The call purpose is: ', call_purpose, '. ',
          'The voicemail should be approximately ', CAST(call_duration_seconds AS STRING), ' seconds long when spoken. ',
          'Only include the representative speaking, no customer response. ',
          'Format as: [Rep: message]'
        )
      ELSE
        CONCAT(
          'Generate a realistic phone conversation transcript between a Telco call center representative named ', rep_name, ' from "Brickster Wireless" and a customer. ',
          'The call purpose is: ', call_purpose, '. ',
          'The call outcome was: ', call_outcome, '. ',
          'The conversation should be approximately ', CAST(call_duration_seconds AS STRING), ' seconds long when spoken. ',
          'Format as alternating lines with [Rep: ...] and [Customer: ...]'
        )
    END
  ) AS transcript
FROM ({table_name})
"""

# Create temp view and run query
transcript_df.createOrReplaceTempView("filtered_calls_temp")
result_df = spark.sql(transcript_query.format(table_name="filtered_calls_temp"))

display(result_df)

In [0]:
result_df.write.mode("overwrite").saveAsTable("historical_telephonic_outreach_transcripts")

In [0]:
COC_Scorecard_Prompt = """
ONLY RETURN ANSWERS IN JSON STRUCT DEFINED BELOW

You are an analyst scoring how well call center support specialists are performing.  You will be given a transcript of a call and the following criteria to judge it by. 

Each criteria has skills within them that you associate a score with.

Criteria 1: Technical Aspects
1. The agent satisfied proper recording disclosure requirements throughout the call.
  - If yes, then they get six points
  - If no, then they get zero points
2. The agent satisfied proper member authentication requirements 
  - If yes, then they get eight points
  - If no, then they get zero points
3. The agent satisfied call closing requirements appropriately 
  - If yes, then they get four points
  - If no, then they get zero points

Criteria 2: Quality of Service
1. The agent maintained professionalism and employed soft skills throughout the call
  - Rate on a scale of 1 to 10 where 1 is unprofessional and 10 is most professional
2. The agent offered correct and accurate program information when necessary
  - Rate on a scale of 1 to 10 with 1 being incorrect and 10 being correct
3. The agent demonstrated a positive and helpful demeanor throughout the call
  - Rate on a scale of 1 to 10 with 1 being negative and 10 being positive

For the response, structure it so that it is a JSON object with the following keys:

- "criteria_1": a dictionary with the following keys:
  - "technical_aspects": a dictionary with the following keys:
    - "recording_disclosure": a dictionary with the following keys:
      - "score": an integer between 0 and 6
    - "member_authentication": a dictionary with the following keys:
      - "score": an integer between 0 and 8
    - "call_closing": a dictionary with the following keys:
      - "score": an integer between 0 and 4
- "criteria_2": a dictionary with the following keys:
  - "quality_of_service": a dictionary with the following keys:
    - "professionalism": a dictionary with the following keys:
      - "score": an integer between 1 and 10
    - "program_information": a dictionary with the following keys:
      - "score": an integer between 1 and 10
    - "demeanor": a dictionary with the following keys:
      - "score": an integer between 1 and 10
- "total_score": an integer between 0 and 48

Your response should ONLY be a JSON object with the above structure.
"""

returnType = """
STRUCT<
  criteria_1: STRUCT<
    technical_aspects: STRUCT<
      recording_disclosure: STRUCT<score: INT>,
      member_authentication: STRUCT<score: INT>,
      call_closing: STRUCT<score: INT>
    >
  >,
  criteria_2: STRUCT<
    quality_of_service: STRUCT<
      professionalism: STRUCT<score: INT>,
      program_information: STRUCT<score: INT>,
      demeanor: STRUCT<score: INT>
    >
  >,
  total_score: INT
>
"""

In [0]:
## improvement: structured outputs
transcription_scored_df = spark.sql(f"""
SELECT 
  *,
  ai_query(
    'databricks-gpt-oss-20b',
    '{COC_Scorecard_Prompt}\nTranscript: ' || transcript
  ) AS scorecard
FROM historical_telephonic_outreach_transcripts
""")
# transcription_scored_df.write.mode("overwrite").saveAsTable("historical_telephonic_outreach_transcripts_scored")

In [0]:
# from pyspark.sql.functions import from_json

# display(transcription_scored_df.withColumn("scorecard", from_json("scorecard", returnType)))

In [0]:
from pyspark.sql.functions import from_json, col

scored_with_json_df = transcription_scored_df.withColumn("scorecard_json", from_json("scorecard", returnType)).drop("scorecard")
scored_with_json_df = scored_with_json_df.withColumn("total_score", col("scorecard_json.total_score"))
scored_with_json_df.write.option("mergeSchema", "true").mode("overwrite").saveAsTable("call_center_scores")

In [0]:
summary_df = spark.sql("""
SELECT 
    *, 
    ai_summarize(transcript) AS transcript_summary
FROM call_center_scores
""")

display(summary_df)

In [0]:
summary_df.write.option("mergeSchema", "true").mode("overwrite").saveAsTable("call_center_scores")

In [0]:
%sql
ALTER TABLE call_center_scores
SET TBLPROPERTIES (delta.enableChangeDataFeed = true)